In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, auc,roc_curve
from sklearn.metrics import precision_score
from sklearn import preprocessing
from sklearn import metrics
from scipy.stats import ttest_ind
import time
from datetime import date
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
from sklearn import svm
import scipy

In [2]:
df = pd.read_csv('train.csv', parse_dates=['last_review', 'host_since'])
df1 = df.copy()

In [3]:
features_mean = list(df.columns[1:24])
df.columns.values.tolist()
feat = [0,4,7,8,18,10,5,9,11,12,15]
sort= [5,9]
nah = [10,11,12,15]
feature_list = []
for i in feat:
    feature_list += [features_mean[i]]

In [4]:
def encodeval(dataframe,features_mean,col):
    dataF = dataframe.copy()
    le = preprocessing.LabelEncoder()
    le.fit(dataF[features_mean[col]])
    dataF[features_mean[col]] = le.transform(dataF[features_mean[col]])
    return dataF

df1['last_review'] = df['last_review'].dt.month
df['host_since'] = pd.to_datetime(df['host_since']).dt.date
df1['host_since'] = -(df['host_since']- date(2020, 10, 5)).dt.days
df1 = encodeval(df1,features_mean,0)
df1 = encodeval(df1,features_mean,1)
df1 = encodeval(df1,features_mean,9)
df1 = encodeval(df1,features_mean,13)
df1 = encodeval(df1,features_mean,18)
df1 = encodeval(df1,features_mean,19)
df1 = encodeval(df1,features_mean,20)
df1 = encodeval(df1,features_mean,21)
df1 = encodeval(df1,features_mean,22)
features_mean.pop(19)

'is_business_travel_ready'

In [5]:
X = df1.loc[:,features_mean]
#X = (X.iloc[:,0:] - X.iloc[:,0:].mean()) / X.iloc[:,0:].std() #standardise
y = df1.loc[:, 'price']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [39]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

In [7]:
dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_val, y_val, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

[18:11:14] 6486x22 matrix with 108244 entries loaded from dtrain.svm
[18:11:14] 3195x22 matrix with 53344 entries loaded from dtest.svm


In [8]:
param = {
    'max_depth': 10,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 5}  # the number of classes that exist in this datset
num_round = 100  # the number of training iterations

In [9]:
bst = xgb.train(param, dtrain, num_round)

In [10]:
bst.dump_model('dump.raw.txt')

In [11]:
preds = bst.predict(dtest)

In [12]:
best_preds = np.asarray([np.argmax(line) for line in preds])

In [13]:
print(precision_score(y_val, best_preds, average='macro'))

0.5415679957137921


In [14]:
print(accuracy_score(y_val, best_preds))

0.5320813771517997


In [15]:
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [16]:
accuracy_score(y_val, y_pred)

0.415962441314554

In [17]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



# creating tensor from targets_df 
X_traintorch = torch.tensor(X_train.values)
y_traintorch = torch.tensor(y_train.values)

X_valtorch = torch.tensor(X_val.values)
y_valtorch = torch.tensor(y_val.values)



In [18]:
class SimpleNet(nn.Module):
    def __init__(self, num_features, num_class):
        super(SimpleNet, self).__init__()
        self.layer_1 = nn.Linear(num_features, num_features//2)
        self.layer_out = nn.Linear(num_features//2, num_class) 
        
        
        #self.lr = nn.LeakyReLU()
        self.bn = nn.BatchNorm1d(num_features//2)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.bn(x)
        #x = self.lr(x)        
        x = self.layer_out(x)
        
        return x

In [38]:
model = SimpleNet(22,5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1)

In [20]:
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    correct = 0
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(X_traintorch.float())
    loss = criterion(outputs, y_traintorch)
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(outputs.data, 1)
    total += y_traintorch.size(0)
    correct += (predicted == y_traintorch).sum().item()
    
    # print statistics
    running_loss += loss.item()
    print('[%d] loss: %.3f acc: %.3f' %
              (epoch + 1, running_loss / 2000, (100 * correct / total)))

print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    outputs = model(X_valtorch.float())
    _, predicted = torch.max(outputs.data, 1)
    total += y_valtorch.size(0)
    correct += (predicted == y_valtorch).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

[1] loss: 0.001 acc: 26.889
[2] loss: 0.002 acc: 32.424
[3] loss: 0.007 acc: 25.270
[4] loss: 0.005 acc: 27.444
[5] loss: 0.002 acc: 33.565
[6] loss: 0.003 acc: 33.703
[7] loss: 0.002 acc: 31.930
[8] loss: 0.001 acc: 25.162
[9] loss: 0.002 acc: 32.593
[10] loss: 0.002 acc: 33.287
[11] loss: 0.001 acc: 28.184
[12] loss: 0.001 acc: 29.911
[13] loss: 0.001 acc: 33.919
[14] loss: 0.001 acc: 33.380
[15] loss: 0.001 acc: 35.045
[16] loss: 0.001 acc: 34.258
[17] loss: 0.001 acc: 33.071
[18] loss: 0.001 acc: 32.100
[19] loss: 0.001 acc: 29.433
[20] loss: 0.001 acc: 32.455
[21] loss: 0.001 acc: 28.014
[22] loss: 0.001 acc: 28.338
[23] loss: 0.001 acc: 34.382
[24] loss: 0.001 acc: 38.282
[25] loss: 0.001 acc: 38.791
[26] loss: 0.001 acc: 33.179
[27] loss: 0.001 acc: 29.972
[28] loss: 0.001 acc: 31.915
[29] loss: 0.001 acc: 38.637
[30] loss: 0.001 acc: 38.529
[31] loss: 0.001 acc: 38.683
[32] loss: 0.001 acc: 39.130
[33] loss: 0.001 acc: 33.642
[34] loss: 0.001 acc: 32.809
[35] loss: 0.001 acc: 3

[287] loss: 0.001 acc: 46.963
[288] loss: 0.001 acc: 47.533
[289] loss: 0.001 acc: 47.040
[290] loss: 0.001 acc: 45.606
[291] loss: 0.001 acc: 46.778
[292] loss: 0.001 acc: 46.377
[293] loss: 0.001 acc: 47.641
[294] loss: 0.001 acc: 45.359
[295] loss: 0.001 acc: 47.209
[296] loss: 0.001 acc: 46.392
[297] loss: 0.001 acc: 47.348
[298] loss: 0.001 acc: 46.639
[299] loss: 0.001 acc: 47.518
[300] loss: 0.001 acc: 46.099
[301] loss: 0.001 acc: 46.207
[302] loss: 0.001 acc: 47.471
[303] loss: 0.001 acc: 47.441
[304] loss: 0.001 acc: 47.749
[305] loss: 0.001 acc: 46.068
[306] loss: 0.001 acc: 47.626
[307] loss: 0.001 acc: 47.595
[308] loss: 0.001 acc: 47.811
[309] loss: 0.001 acc: 46.916
[310] loss: 0.001 acc: 47.286
[311] loss: 0.001 acc: 47.980
[312] loss: 0.001 acc: 46.670
[313] loss: 0.001 acc: 47.271
[314] loss: 0.001 acc: 47.749
[315] loss: 0.001 acc: 45.822
[316] loss: 0.001 acc: 46.408
[317] loss: 0.001 acc: 46.809
[318] loss: 0.001 acc: 46.469
[319] loss: 0.001 acc: 46.916
[320] loss

[569] loss: 0.005 acc: 38.159
[570] loss: 0.004 acc: 23.096
[571] loss: 0.003 acc: 27.552
[572] loss: 0.002 acc: 30.034
[573] loss: 0.003 acc: 24.561
[574] loss: 0.002 acc: 31.946
[575] loss: 0.003 acc: 23.158
[576] loss: 0.004 acc: 26.565
[577] loss: 0.004 acc: 34.073
[578] loss: 0.004 acc: 38.483
[579] loss: 0.003 acc: 31.360
[580] loss: 0.003 acc: 28.662
[581] loss: 0.002 acc: 19.087
[582] loss: 0.002 acc: 30.558
[583] loss: 0.004 acc: 31.129
[584] loss: 0.003 acc: 26.565
[585] loss: 0.003 acc: 28.708
[586] loss: 0.005 acc: 28.893
[587] loss: 0.008 acc: 30.142
[588] loss: 0.004 acc: 38.514
[589] loss: 0.004 acc: 37.974
[590] loss: 0.002 acc: 34.952
[591] loss: 0.002 acc: 26.688
[592] loss: 0.003 acc: 26.596
[593] loss: 0.002 acc: 36.849
[594] loss: 0.003 acc: 35.137
[595] loss: 0.002 acc: 35.091
[596] loss: 0.001 acc: 24.715
[597] loss: 0.002 acc: 26.334
[598] loss: 0.001 acc: 26.426
[599] loss: 0.001 acc: 38.190
[600] loss: 0.002 acc: 37.172
[601] loss: 0.001 acc: 25.100
[602] loss

[845] loss: 0.001 acc: 40.888
[846] loss: 0.001 acc: 42.029
[847] loss: 0.001 acc: 42.661
[848] loss: 0.001 acc: 42.507
[849] loss: 0.001 acc: 42.245
[850] loss: 0.001 acc: 42.276
[851] loss: 0.001 acc: 42.199
[852] loss: 0.001 acc: 41.998
[853] loss: 0.001 acc: 42.414
[854] loss: 0.001 acc: 42.291
[855] loss: 0.001 acc: 42.322
[856] loss: 0.001 acc: 42.584
[857] loss: 0.001 acc: 42.754
[858] loss: 0.001 acc: 42.245
[859] loss: 0.001 acc: 42.476
[860] loss: 0.001 acc: 42.692
[861] loss: 0.001 acc: 42.738
[862] loss: 0.001 acc: 42.985
[863] loss: 0.001 acc: 42.553
[864] loss: 0.001 acc: 42.569
[865] loss: 0.001 acc: 43.139
[866] loss: 0.001 acc: 42.877
[867] loss: 0.001 acc: 42.815
[868] loss: 0.001 acc: 42.969
[869] loss: 0.001 acc: 42.862
[870] loss: 0.001 acc: 43.278
[871] loss: 0.001 acc: 43.232
[872] loss: 0.001 acc: 43.108
[873] loss: 0.001 acc: 43.386
[874] loss: 0.001 acc: 43.525
[875] loss: 0.001 acc: 43.478
[876] loss: 0.001 acc: 43.586
[877] loss: 0.001 acc: 43.725
[878] loss

In [21]:
XGB = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, early_stopping=5, silence=False)
XGB.fit(X_train, y_train)
y_pred = XGB.predict(X_val)

print(accuracy_score(y_val, np.around(y_pred)))

[18:11:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.49827856025039124


In [22]:
np.around(y_pred)

array([2., 1., 3., ..., 1., 1., 2.], dtype=float32)

In [23]:
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LinearRegression
reg=LogisticRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_val)

print(accuracy_score(y_val,y_pred ))

0.37996870109546166


C:\Users\sayan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [24]:
from sklearn.tree import DecisionTreeRegressor
DTree=DecisionTreeRegressor()
DTree.fit(X_train,y_train)
y_pred=DTree.predict(X_val)

print(accuracy_score(y_val,y_pred ))

0.442566510172144


In [25]:
import lightgbm as lgb

In [26]:
train_data=lgb.Dataset(X_train,label=y_train)
param = {'num_leaves':150, 'max_depth':7,'learning_rate':.05,'max_bin':200}

val_data=lgb.Dataset(X_val,label=y_val)
clf = lgb.train(param,train_data,100)
y_pred=clf.predict(X_val)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000971 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1436
[LightGBM] [Info] Number of data points in the train set: 6486, number of used features: 22
[LightGBM] [Info] Start training from score 2.491520
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [27]:
print(accuracy_score(y_val,np.around(y_pred)))

0.49295774647887325


In [28]:
(.12*0.4*0.4)/(.64*.6*.6 + .08*.6*.4 + .16*.6*.4 + .12*.4*.4)

0.06250000000000001

In [29]:
(.64*.6*.6 + 0.16*.6*.4)*.75/.2304

0.875

In [30]:
(.08*.6*.4 + 0.12*.4*.4)*.0625/(.08*.4*.6)

0.12500000000000003

In [31]:
(.64*.6*.6 + 0.08*.6*.4)*.75/.2304

0.8124999999999999

In [32]:
(.16*.6*.4 + 0.12*.4*.4)*.125/(.16*.4*.6)

0.18750000000000003

In [33]:
0.0384/(.64*.6*.6 + .08*.6*.4 + .16*.6*.4 + .12*.4*.4)

0.125

In [34]:
X.iloc[:,19] 

0       2
1       1
2       1
3       0
4       1
       ..
9676    2
9677    2
9678    0
9679    1
9680    1
Name: cancellation_policy, Length: 9681, dtype: int32

In [35]:
X = df1.loc[:,features_mean]

In [36]:
features_mean.pop(19)

'cancellation_policy'

In [37]:
X

,neighbourhood,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,host_since,host_is_superhost,...,beds,bed_type,cleaning_fee,guests_included,extra_people,maximum_nights,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,21,0,1,170,5,2.48,4,346,2622,1,...,1,4,2092,1,0,365,1,2,0,0
1,9,2,1,11,7,0.57,1,0,2425,1,...,1,4,558,1,0,15,0,1,0,0
2,27,0,3,2,11,0.08,19,360,1082,0,...,1,4,1052,3,351,90,1,1,0,0
3,27,0,2,1,11,0.13,1,267,612,0,...,3,4,0,4,0,1125,1,0,0,0
4,31,0,2,31,12,1.54,5,365,2081,1,...,3,4,2092,1,0,365,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9676,26,0,7,7,6,0.89,2,328,467,0,...,1,4,700,1,0,185,0,2,0,0
9677,17,0,1,5,1,0.43,1,0,3403,1,...,5,4,1500,6,500,30,1,2,0,0
9678,21,0,3,1,3,0.30,28,174,2324,1,...,2,4,3492,1,0,180,1,0,0,0
9679,27,0,2,28,4,0.58,1,0,1615,0,...,3,4,900,1,0,1125,1,1,0,0
